<font color="#CA3532"><h1 align="left">Master Data Analytics. EDEM.</h1></font>
<font color="#6E6E6E"><h2 align="left">Herramientas MLOps.</h2></font>
<font color="#6E6E6E"><h2 align="left">Tarea 1. Pipeline entrenamiento de modelos.</h2></font>
#### Daniel Ruiz Riquelme
https://docs.metaflow.org/metaflow/basics

##  Install dependencies

In [1]:
!pip install metaflow
!pip install scikit-learn
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 825.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.8 MB/s eta 0:00:00


## Set username

In [2]:
# Set username for workflows
import os
os.environ["USERNAME"] = "rafa"

In [3]:
%%writefile metaflow_trainingflow.py
from metaflow import FlowSpec, Parameter, step
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import pickle

class TrainingFlow(FlowSpec):
    # Define the parameters for the flow
    max_depth = Parameter('max_depth', default=2, help='Max depth of the random forest classifier')
    n_estimators = Parameter('n_estimators', default=100, help='Number of estimators for the random forest classifier')
    random_state = Parameter('random_state', default=0, help='Random state for the random forest classifier')

    @step
    def start(self):
        # Start the flow
        self.next(self.ingest_data)

    @step
    def ingest_data(self):
        from sklearn.datasets import load_iris

        # Load the iris dataset
        iris = load_iris()

        #pylint: disable=no-member
        self.X = iris.data
        self.y = iris.target
        #pylint: enable=no-member

        self.next(self.split_data)

    @step
    def split_data(self):
        #Split the data into train and test
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=self.random_state
        )

        self.next(self.train)

    @step
    def train(self):
        # Train the model
        self.model = RandomForestClassifier(
            max_depth=self.max_depth,
            n_estimators=self.n_estimators,
            random_state=self.random_state
        )

        self.model.fit(self.X_train, self.y_train)

        self.next(self.show_metrics)

    @step
    def show_metrics(self):
        # Print some metrics
        self.y_pred = self.model.predict(self.X_test)

        self.accuracy = accuracy_score(self.y_test, self.y_pred)

        print(f'Accuracy: {self.accuracy}')
        print('Classification Report:')
        print(classification_report(self.y_test, self.y_pred))

        self.next(self.register_model)

    @step
    def register_model(self):
        # Save the model in a pickle file in local storage
        with open('random_forest_model.pkl', 'wb') as f:
            pickle.dump(self.model, f)

        print("Model saved as random_forest_model.pkl")

        self.next(self.end)

    @step
    def end(self):
        pass

if __name__ == '__main__':
    TrainingFlow()

Writing metaflow_trainingflow.py


In [4]:
!python metaflow_trainingflow.py run --max_depth 2 --n_estimators 100 --random_state 0

Metaflow 2.12.5 executing TrainingFlow for user:rafa
Creating local datastore in current directory (/content/.metaflow)
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint not found, so extra checks are disabled.
2024-06-22 10:40:39.727 Workflow starting (run-id 1719052839726376):
2024-06-22 10:40:39.737 [1719052839726376/start/1 (pid 2292)] Task is starting.
2024-06-22 10:40:40.874 [1719052839726376/start/1 (pid 2292)] Task finished successfully.
2024-06-22 10:40:40.879 [1719052839726376/ingest_data/2 (pid 2306)] Task is starting.
2024-06-22 10:40:42.461 [1719052839726376/ingest_data/2 (pid 2306)] Task finished successfully.
2024-06-22 10:40:42.467 [1719052839726376/split_data/3 (pid 2316)] Task is starting.
2024-06-22 10:40:44.229 [1719052839726376/split_data/3 (pid 2316)] Task finished successfully.
2024-06-22 10:40:44.235 [1719052839726376/train/4 (pid 2330)] Task is starting.
2024-06-22 10:40:45.857 [1719052839726376/train/4 (pid 2330)] Task finished suc